# Watson NLU in IBM Cloud Pak For Data 3.0 for Conversation Analysis of a Watson Assistant Chatbot

The purpose of this notebook is to demo how to extract logs using the Watson Assistant API, authenticate and call the NLU API for conversation log analysis, and enhance other analytical tools for visual monitoring of the chatbot post production. 

## Table of Contents

* [Extract Conversation Logs from Watson Assistant](#wa)
* [Authenticate Watson NLU API](#nlu)
* [Analyze Conversation Logs](#acl)
* [Monitor Chatbot Performance](#mcp)


## Extract Conversation Logs From Watson Assistant <a class="anchor" id="wa"></a>

In this section, you will be using the Watson Assistant API to extract conversation logs in a JSON file.  First, you will be authenticating the API, in which you will need to enter the required credentials.  Then, you will be calling on a request to list the log events in the workspace specified.

#### Credentials Required:
* API Key of your Watson Assistant Chatbot
* URL of your Watson Assistant Chatbot
* Workspace ID, also identified as Skill ID


In [1]:
!pip install --upgrade "ibm-watson>=4.5.0"

     |████████████████████████████████| 370 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 13.2 MB/s eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-4.5.0-py3-none-any.whl size=364300 sha256=0ba7eb8d5352103aff87b12026c5bf392bde98d335fc627a7c8c43eef5a0a9cd
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b3/ad/56/41cba4fcf008f2ec95d26ce20e11587d35924d0d4d133d830f
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-1.5.1-py3-none-any.whl size=44489 sha256=5261e4a6a53e84621dba28ef9494cf987b3916883d81e5eb7f72c7a0b2f4d9a6
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/d6/9b/83/65fcb585e0ab9b448be7b13049e21b3432224782198bcc9673
Successfully built ibm-watson ibm-cloud-sdk-core


In [3]:
#Import the following packages needed to extract Watson Assistant conversation logs
import json
#Use AssistantV2 if your Watson Assistant chatbot is on a Cloud Pak for Data Environment with WA V2 API
from ibm_watson import AssistantV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

authenticator = IAMAuthenticator('YOUR_API_KEY') 
assistant = AssistantV1(
    version='',
    authenticator = authenticator
)

assistant.set_service_url('Your URL') 

In [4]:
#List log events in a workspace 
response=assistant.list_logs(
    workspace_id='YOUR_SKILL_ID'  
).get_result()

json_log = (json.dumps(response, indent=2))
#print(json_log)

## Authenticate Watson NLU API <a class="anchor" id="nlu"></a>

Now that you have extracted the logs of your Watson Assistant chatbot, you will authenticate the Watson NLU service instance API in the following section.

#### Credentials Required:

* Your username for the IBM Cloud Pak for Data 3.0 environment (This should have been provided to you by your system administrator)
* Your password for the IBM Cloud Pak for Data 3.0 environment
* The URL of the IBM Cloud Pak for Data 3.0 environment, in which the Watson NLU service instance is provisioned
* The NLU service instance URL
    * This can be found in the connection details section of the service instance details page or in the credentials file you downloaded when you provisioned the instance. 

In [17]:
#Import required packages to call the NLU API
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator


authenticator = CloudPakForDataAuthenticator(
    'YOUR_USERNAME', 
    'YOUR_PASSWORD',
    'URL', 
    disable_ssl_verification=True   #False for customer environments
)

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='',
    authenticator=authenticator
)

natural_language_understanding.set_service_url('NLU_SERVICE_INSTANCE_URL') 

natural_language_understanding.set_disable_ssl_verification(True) #False for customer environments

## Analyze JSON file of Conversation Logs <a class="anchor" id="acl"></a>

Once you have successfully authenticated the Watson NLU API, you can call on the following request to analyze the text from the conversation logs

In [23]:
#Import requires pacakges to analyze conversation logs
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, EmotionOptions, EntitiesOptions 
from ibm_watson.natural_language_understanding_v1 import KeywordsOptions, SentimentOptions, RelationsOptions

response = natural_language_understanding.analyze(
        text=json_log,
        features = Features(sentiment = SentimentOptions(document=True),
                            keywords=KeywordsOptions(), 
                            categories=CategoriesOptions(),
                            entities=EntitiesOptions(),
                            emotion=EmotionOptions(),
                            relations=RelationsOptions()
                           )).get_result()

print(json.dumps(response, indent=2))

/opt/conda/envs/Python-3.6-WMLCE/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


{
  "usage": {
    "text_units": 5,
    "text_characters": 50000,
    "features": 3
  },
  "sentiment": {
    "document": {
      "score": -0.327048,
      "mixed": "1",
      "label": "negative"
    }
  },
  "language": "en",
  "keywords": [
    {
      "text": "bc2b0ea",
      "relevance": 0.642827,
      "count": 4
    },
    {
      "text": "coffee shop website",
      "relevance": 0.616147,
      "count": 8
    },
    {
      "text": "IBMuid-3d353dcc-09ee",
      "relevance": 0.614215,
      "count": 4
    },
    {
      "text": "8eb3-5e5cf709dcdb",
      "relevance": 0.56867,
      "count": 4
    },
    {
      "text": "1a0fcac5",
      "relevance": 0.56545,
      "count": 22
    },
    {
      "text": "cold brew",
      "relevance": 0.56063,
      "count": 2
    },
    {
      "text": "31115-e2b0-4c38",
      "relevance": 0.554258,
      "count": 10
    },
    {
      "text": "main skill",
      "relevance": 0.552681,
      "count": 22
    },
    {
      "text": "variety of hot 

## Monitor Chatbot Performance <a class="anchor" id="mcp"></a>

In this section, you will learn how to monitor chatbot performance using analytical visuals, such as a heat map.

In [ ]:
#Add Heatmap
#Satisfaction rating, emotion score